In [1]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 0s (5,639 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading

In [2]:
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install huggingface_sb3
!pip install ale-py==0.7.4 # To overcome an issue with gym (https://github.com/DLR-RM/stable-baselines3/issues/875)

     |████████████████████████████████| 448 kB 12.8 MB/s 
     |████████████████████████████████| 177 kB 12.8 MB/s 
     |████████████████████████████████| 1.5 MB 33.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=50e745f58385ac463d71614f99b7b5a85b8523ab5748c0dd5d7db522eec99e94
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=ab67ee98cf0ea2aba535f93309659979ae0f49e00aa1712f746c5e490ea3db8d
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found ex

In [3]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [4]:
# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, done and info
  observation, reward, done, info = env.step(action)
  
  # If the game is done (in our case we land, crashed or timeout)
  if done:
      # Reset the environment
      print("Environment is reset")
      observation = env.reset()

Action taken: 1
Action taken: 0
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 3


In [5]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-2.665266   -0.20614141  0.21451198  1.9937919   0.8112593  -1.1282082
 -1.8042814  -0.0321768 ]


In [9]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [10]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

In [ ]:
# TODO: Define a PPO MlpPolicy architecture
# We use MultiLayerPerceptron (MLPPolicy) because the input is a vector,
# if we had frames as input we would use CnnPolicy
model = PPO('MlpPolicy', env, verbose=1)

Using cuda device


In [13]:
# We added some parameters to fasten the training
model_1 = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


In [ ]:
model_1.learn(total_timesteps=1000000)

In [15]:
model_1.get_env()

In [17]:
# TODO: Evaluate the agent
# Create a new environment for evaluation
eval_env = gym.make('LunarLander-v2')

# Evaluate the model with 10 evaluation episodes and deterministic=True
mean_reward, std_reward = evaluate_policy(model_1, model_1.get_env(), n_eval_episodes=10)

# Print the results
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")


mean_reward=257.00 +/- 19.427065330429766


In [19]:
model_1.save("ppo-LunarLander-kharitonov_v2")

In [20]:
from huggingface_hub import notebook_login

In [21]:
notebook_login()
!git config --global credential.helper store

Login successful
Your token has been saved to /root/.huggingface/token


In [22]:
repo_id = 'KhariotnovKK/luna_lender_v1'
commit_message = "number of iterations increased to 1mln"
model_name = 'Luna_lender_kharitonov_1mln'
env_id = "LunarLander-v2"
model_architecture = "PPO"

In [23]:
from stable_baselines3.common.vec_env import DummyVecEnv

In [24]:
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

In [26]:
package_to_hub(model=model_1, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: If you encounter a bug, please open an issue and use
push_to_hub instead.


/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:82: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.7. Pass `repo_id` instead.
  FutureWarning,
Cloning https://huggingface.co/KhariotnovKK/luna_lender_v1 into local empty directory.
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Saving video to /content/-step-0-to-step-1000.mp4
ℹ Pushing repo luna_lender_v1 to the Hugging Face Hub


Upload file replay.mp4:   1%|1         | 3.34k/240k [00:00<?, ?B/s]

Upload file Luna_lender_kharitonov_1mln.zip:   2%|2         | 3.34k/141k [00:00<?, ?B/s]

Upload file Luna_lender_kharitonov_1mln/policy.pth:   8%|7         | 3.34k/42.2k [00:00<?, ?B/s]

Upload file Luna_lender_kharitonov_1mln/policy.optimizer.pth:   4%|4         | 3.34k/82.8k [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/KhariotnovKK/luna_lender_v1
   edc401e..e2fa443  main -> main



ℹ Your model is pushed to the hub. You can view your model here:
https://huggingface.co/KhariotnovKK/luna_lender_v1


'https://huggingface.co/KhariotnovKK/luna_lender_v1'

# Car racing environment experiment

In [4]:
env = gym.make("CarRacing-v0")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
# print("Sample observation", env.observation_space.sample()) # Get a random observation

Track generation: 1108..1394 -> 286-tiles track
_____OBSERVATION SPACE_____ 

Observation Space Shape (96, 96, 3)


In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()


from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=60):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)


class trigger:
    def __init__(self):
        self._trigger = True

    def __call__(self, e):
        return self._trigger

    def set(self, t):
        self._trigger = t

In [8]:
frames = []
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        frames.append(env.render(mode='rgb_array'))
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode,score))
env.close()

Track generation: 917..1156 -> 239-tiles track
Episode:1 Score:-20.168067226890653
Track generation: 1102..1387 -> 285-tiles track
Episode:2 Score:-29.577464788732808
Track generation: 1103..1383 -> 280-tiles track
Episode:3 Score:-28.315412186380247
Track generation: 1232..1543 -> 311-tiles track
Episode:4 Score:-35.483870967742455
Track generation: 1188..1489 -> 301-tiles track
Episode:5 Score:-33.33333333333377
Track generation: 1415..1773 -> 358-tiles track
Episode:6 Score:-43.977591036415305
Track generation: 1152..1444 -> 292-tiles track
Episode:7 Score:-31.271477663230627
Track generation: 1072..1344 -> 272-tiles track
Episode:8 Score:-29.88929889298926
Track generation: 1131..1418 -> 287-tiles track
Episode:9 Score:-30.06993006993051
Track generation: 1195..1497 -> 302-tiles track
Episode:10 Score:-36.87707641196071


In [33]:
env.action_space

Box([-1.  0.  0.], [1. 1. 1.], (3,), float32)

In [34]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape Box([-1.  0.  0.], [1. 1. 1.], (3,), float32)
Action Space Sample [-0.25699452  0.14605676  0.4581043 ]


In [5]:
# Create the environment
env = make_vec_env('CarRacing-v0', n_envs=16)

In [6]:
# We added some parameters to fasten the training
model_car = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 24,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:141: UserWarning: You have specified a mini-batch size of 24, but because the `RolloutBuffer` is of size `n_steps * n_envs = 16384`, after every 682 untruncated mini-batches, there will be a truncated mini-batch of size 16
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=1024 and n_envs=16)
  f"You have specified a mini-batch size of {batch_size},"


In [ ]:
model_car.learn(total_timesteps=500000)

Track generation: 1076..1356 -> 280-tiles track
Track generation: 1118..1398 -> 280-tiles track
Track generation: 979..1235 -> 256-tiles track
Track generation: 987..1243 -> 256-tiles track
Track generation: 1080..1361 -> 281-tiles track
Track generation: 1177..1475 -> 298-tiles track
Track generation: 1144..1434 -> 290-tiles track
Track generation: 1167..1463 -> 296-tiles track
Track generation: 1152..1452 -> 300-tiles track
Track generation: 1125..1410 -> 285-tiles track
Track generation: 1163..1458 -> 295-tiles track
Track generation: 1084..1359 -> 275-tiles track
Track generation: 992..1243 -> 251-tiles track
Track generation: 1048..1314 -> 266-tiles track
Track generation: 1096..1382 -> 286-tiles track
Track generation: 1256..1574 -> 318-tiles track
Track generation: 1141..1431 -> 290-tiles track
Track generation: 971..1223 -> 252-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1034..1302 -> 268-tiles track
Track genera

In [ ]:
# TODO: Evaluate the agent
# Create a new environment for evaluation
eval_env = gym.make('CarRacing-v0')

# Evaluate the model with 10 evaluation episodes and deterministic=True
mean_reward, std_reward = evaluate_policy(model_car, model_car.get_env(), n_eval_episodes=10)

# Print the results
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")


In [ ]:
model_car.save("car_racing_kk_v3")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [19]:
repo_id = 'KhariotnovKK/Car_racing_v0'
commit_message = "increased n_envs and n_steps in model"
model_name = 'car_racing_kk_v0'
env_id = "CarRacing-v0"
model_architecture = "PPO"

In [20]:
from stable_baselines3.common.vec_env import DummyVecEnv
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

In [21]:
package_to_hub(model=model_car, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: If you encounter a bug, please open an issue and use
push_to_hub instead.


/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:82: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.7. Pass `repo_id` instead.
  FutureWarning,
/content/hub/Car_racing_v0 is already a clone of https://huggingface.co/KhariotnovKK/Car_racing_v0. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Track generation: 1095..1381 -> 286-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1127..1413 -> 286-tiles track
Track generation: 1217..1535 -> 318-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1237..1551 -> 314-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1371..1718 -> 347-tiles track
Track generation: 1227..1538 -> 311-tiles track
Track generation: 1196..1499 -> 303-tiles track
Track generation: 1115..1398 -> 283-tiles track
Track generation: 1200..1504 -> 304-tiles track
Track generation: 938..1184 -> 246-tiles track
Track generation: 1112..1394 -> 282-tiles track
Track generation: 1134..1422 -> 288-tiles track
Track generation: 1323..1658 -> 335-tiles track
Saving video to /content/-step-0-to-step-1000.mp4
ℹ Pushing repo Car_racing_v0 to the Hugging Face Hub


Upload file car_racing_kk_v0.zip:   0%|          | 3.34k/41.0M [00:00<?, ?B/s]

Upload file car_racing_kk_v0/policy.optimizer.pth:   0%|          | 3.34k/27.1M [00:00<?, ?B/s]

Upload file replay.mp4:   0%|          | 3.34k/987k [00:00<?, ?B/s]

Upload file car_racing_kk_v0/policy.pth:   0%|          | 3.34k/13.5M [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/KhariotnovKK/Car_racing_v0
   c22f418..f913c56  main -> main



ℹ Your model is pushed to the hub. You can view your model here:
https://huggingface.co/KhariotnovKK/Car_racing_v0


'https://huggingface.co/KhariotnovKK/Car_racing_v0'